# Testing Driver Drowsiness, Detection, Play Sound

In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer  #alert sound
import time

import pickle

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Sound alert if eyes are closed

In [2]:
mixer.init()
alarm_sound = mixer.Sound("audio\groovy_energy.mp3")

## Haar cascades
A Haar cascade classifier is a machine learning object detection program that identifies objects in an image and video.
Haar is an algorithm that is capable of detecting objects in images, regardless of their location and scale in an image
    
Haar cascades have so many models to detect various parts of a human body like to detect eyes, frontal face, lower body, upper body, full body 

In [3]:
face_detection = cv2.CascadeClassifier('haar_cascade\haarcascade_frontalface_alt.xml')
left_eye_detection = cv2.CascadeClassifier('haar_cascade\haarcascade_lefteye_2splits.xml')
right_eye_detection = cv2.CascadeClassifier('haar_cascade\haarcascade_righteye_2splits.xml')
labels =['Close','Open']

#path = "C:\Users\jngai\AppData\Roaming\Python\Python39\site-packages\cv2\data"
#face_detection = cv2.CascadeClassifier("C:\Users\jngai\AppData\Roaming\Python\Python39\site-packages\cv2\data\haarcascade_frontalface_alt.xml")
#left_eye_detection = cv2.CascadeClassifier("C:\Users\jngai\AppData\Roaming\Python\Python39\site-packages\cv2\data\haarcascade_lefteye_2splits.xml")
#right_eye_detection = cv2.CascadeClassifier("C:\Users\jngai\AppData\Roaming\Python\Python39\site-packages\cv2\data\haarcascade_righteye_2splits.xml")

## Loading the created model

In [4]:
# load the model we created

#model = load_model("result_model.h5")
model = load_model("model-014.model")

## Using OpenCV to access the webcam and capture frames

In [5]:
path = os.getcwd()

# capture each frame
capture = cv2.VideoCapture(0)

#check if the webcam is open
if capture.isOpened():
    capture = cv2.VideoCapture(0)
if not capture.isOpened():
    raise IOError("Cannot open webcam")

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

In [6]:
#declaring variables 
counter = 0
time = 0
thick = 2
right_eye_pred=[99]
left_eye_pred=[99]

In [ ]:
while(True):
    ret, frame = capture.read()
    height,width = frame.shape[:2] 

    #convert the captured image to grey color:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #perform detection(this will return x,y coordinates , height , width of the boundary boxes object)
    faces = face_detection.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = left_eye_detection.detectMultiScale(gray)
    right_eye =   right_eye_detection.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (100,height) , (0,0,0) , thickness=cv2.FILLED )
    #cv2.rectangle(frame, (290,height-50) , (540,height) , (0,0,0) , thickness=cv2.FILLED )

    #iterating over faces and drawing boundary boxes for each face:
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        
    #iterating over right eye:
    for (x,y,w,h) in right_eye:
        #pull out the right eye image from the frame:
        right_one=frame[y:y+h,x:x+w]
        counter += 1
        right_one = cv2.cvtColor(right_one,cv2.COLOR_BGR2GRAY)
        right_one = cv2.resize(right_one,(24,24))
        right_one = right_one/255
        right_one =  right_one.reshape(24,24,-1)
        right_one = np.expand_dims(right_one,axis=0)
        #right_eye_pred = model.predict_classes(right_one)
        #right_eye_pred = model.predict(right_one).astype("int32")
        right_eye_pred = np.argmax(model.predict(right_one),axis=1)
        if(right_eye_pred[0] == 1):
            labels = 'Open' 
        if(right_eye_pred[0]==0):
            labels = 'Closed'
        break

    #iterating over left eye:
    for (x,y,w,h) in left_eye:
        #pull out the left eye image from the frame:
        left_one=frame[y:y+h,x:x+w]
        counter += 1
        left_one = cv2.cvtColor(left_one,cv2.COLOR_BGR2GRAY)  
        left_one = cv2.resize(left_one,(24,24))
        left_one = left_one/255
        left_one = left_one.reshape(24,24,-1)
        left_one = np.expand_dims(left_one,axis=0)
        #left_eye_pred = model.predict_classes(left_one)
        #left_eye_pred = model.predict(left_one).astype("int32")
        left_eye_pred = np.argmax(model.predict(left_one),axis=1)
        if(left_eye_pred[0] == 1):
            labels ='Open'   
        if(left_eye_pred[0] == 0):
            labels ='Closed'
        break

    #checking - if the person’s left eye and right eye are closed, time incremenets. If timer count> 10 make alert sound. If both eyes open, timer decrements and sound stops.

    #print(right_eye_pred[0])
    #print("l:",left_eye_pred[0])
    
    if(right_eye_pred[0] == 0 and left_eye_pred[0] == 0):
        time += 1
        cv2.putText(frame,"Inactive",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(right_eye_pred[0]==1 or left_eye_pred[0]==1):
    else:
        time -= 1
        cv2.putText(frame,"Active",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        
    
    if(time<0):
        time=0   
    cv2.putText(frame,'Wake up Time !!:'+str(time),(100,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
    if(time>5):
        #person is falling asleep, therefore alert.
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            alarm_sound.play()
            
            input("Write anything to stop.")
            alarm_sound.stop()
            
        except:  # isplaying = False
            pass
        if(thick < 6):
            thick = thick+2
        else:
            thick=thick-2
            if(thick<2):
                thick=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thick)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break   
capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step


Write anything to stop. q


References:

Haar cascades 
* https://github.com/opencv/opencv/tree/master/data/haarcascades
* https://medium.com/analytics-vidhya/haar-cascades-explained-38210e57970d
* https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf

Other sources:  
https://stackoverflow.com/questions/68836551/keras-attributeerror-sequential-object-has-no-attribute-predict-classes/68841446#68841446https://stackoverflow.com/questions/68836551/keras-attributeerror-sequential-object-has-no-attribute-predict-classes/68841446#68841446


Notes:   
* In this project, we use the haar cascade classifier and a CNN model to predict the status (eyes closed or open).

In [ ]:
extra:
    
https://data-flair.training/blogs/python-project-driver-drowsiness-detection-system/

#Sendgrid
username: julesbabej@gmail.com
password: babej01234567890